In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Пример задачи<a class="anchor" id="example"></a><center>

__Задача:__ предсказание баллов ЕГЭ ученика в зависимости от кол-ва лет стажа его репетитора

In [ ]:
X = np.array([[ 1,  1],
              [ 1,  1],
              [ 1,  2],
              [ 1,  5],
              [ 1,  3],
              [ 1,  0],
              [ 1,  5],
              [ 1, 10],
              [ 1,  1],
              [ 1,  2]])

In [ ]:
X.shape

In [ ]:
y = [45, 55, 50, 55, 60, 35, 75, 80, 50, 60]

In [ ]:
plt.scatter(X[:, 1], y);

Уравнение прямой: $y = a*x + b$

In [ ]:
y_pred1 = 5 * X[:, 1] + 35 * X[:, 0]
y_pred2 = 7.5 * X[:, 1] + 40 * X[:, 0]

In [ ]:
plt.scatter(X[:, 1], y)
plt.plot(X[:, 1], y_pred1, label='1')
plt.plot(X[:, 1], y_pred2, label='2')
plt.legend()
plt.show()

Отклонение

In [ ]:
err1 = np.sum(y - y_pred1)
err2 = np.sum(y - y_pred2)
err1, err2

MAE (Mean Absolute Error)

In [ ]:
mae_1 = np.mean(np.abs(y - y_pred1))
mae_2 = np.mean(np.abs(y - y_pred2))
mae_1, mae_2

MSE (Mean Squared Error)

In [ ]:
mse_1 = np.mean((y - y_pred1)**2)
mse_2 = np.mean((y - y_pred2)**2)
mse_1, mse_2

### Метод наименьших квадратов (МНК)

$$w = (X^{T}X)^{-1}X^{T}y.$$


In [ ]:
W_analytical = np.linalg.inv(np.dot(X.T, X)) @ X.T @ y
W_analytical

In [ ]:
y_pred_analytical = W_analytical[0] * X[:, 0] + W_analytical[1] * X[:, 1]
y_pred_analytical = X @ W_analytical

In [ ]:
plt.scatter(X[:, 1], y)
plt.plot(X[:, 1], y_pred1, label='1 - manual')
plt.plot(X[:, 1], y_pred2, label='2 - manual')
plt.plot(X[:, 1], y_pred_analytical, label='3 - analytical solution')
plt.legend()
plt.show()

In [ ]:
def calc_mae(y, y_pred):
    err = np.mean(np.abs(y - y_pred))
    return err

def calc_mse(y, y_pred):
    err = np.mean((y - y_pred)**2)
    return err

In [ ]:
calc_mae(y, y_pred1), calc_mse(y, y_pred1)

In [ ]:
calc_mae(y, y_pred2), calc_mse(y, y_pred2)

In [ ]:
calc_mae(y, y_pred_analytical), calc_mse(y, y_pred_analytical)

### Градиентный спуск

$$\nabla_{w}Q(w,X) = \frac{2}{l}X^{T}(Xw-y).$$

1. Инициализация w

2. Цикл по k = 1,2,3,...:

    * $w^{k} = w^{k-1} - \eta_{k}\nabla Q(w^{k-1}, X)$

    * Если $||w^{k} - w^{k-1}|| < \epsilon$, то завершить.


In [ ]:
W = np.random.normal(size=(X.shape[1]))
W

In [ ]:
eta = 0.02 # величина шага

In [ ]:
X.shape,  W.shape

In [ ]:
n = len(y)
dQ = 2/n * X.T @ (X @ W - y) # градиент функции ошибки
dQ

In [ ]:
grad = eta * dQ
grad

In [ ]:
print(f'previous weights', W)
W = W - grad
print(f'new weights', W)

In [ ]:
y_pred_grad = X @ W
plt.scatter(X[:, 1], y)
plt.plot(X[:, 1], y_pred_grad, label='gradient descent', c='g')
plt.legend()
plt.show()

### Домашнее задание <a class="anchor" id="hw"></a><center>

1. Подберите скорость обучения (eta) и количество итераций

In [ ]:
n = X.shape[0]
#Изменяю значение eta и n_iter. Подбирал сначала меньше стартового значения, MSE была больше стартовой. Затем поставил 0, это уменьшило mse, потом попробовал 0.1 это дало минимальную MSE. Попробовал следом увеличивать значение, поставил 0.2. При таком значении MSE уже растет, нам этого не нужно.
#Исходя из этого рассуждения посчитал что оптимальная eta = 0.1. Итераций я поставил сначала 10000 и пошел проверять, обнаружил что после 120й итерации цифра не меняется у -2 разряда. Оставил оптимальное значение 120 итераций
eta = 0.1
n_iter = 120

W = np.array([1, 0.5])
print(f'Number of objects = {n} \
       \nLearning rate = {eta} \
       \nInitial weights = {W} \n')

for i in range(n_iter):
    y_pred = np.dot(X, W)
    err = calc_mse(y, y_pred)
    for k in range(W.shape[0]):
        W[k] -= eta * (1/n * 2 * X[:, k] @ (y_pred - y))
    if i % 10 == 0:
        eta /= 1.1
        print(f'Iteration #{i}: W_new = {W}, MSE = {round(err, 2)}')

2*. В этом коде мы избавляемся от итераций по весам, но тут есть ошибка, исправьте ее

In [ ]:
n = X.shape[0]
#По принципу из первого примера, перекалибрую эпсилон и итерации
eta = 1e-2
n_iter = 700

W = np.array([1, 0.5])
print(f'Number of objects = {n} \
       \nLearning rate = {eta} \
       \nInitial weights = {W} \n')

for i in range(n_iter):
    y_pred = np.dot(X, W)
    err = calc_mse(y, y_pred)
    #Чтобы решить ошибку матрицу X надо транспонировать. Нельзя матрицу (10, 2) перемножить на вектор (1, 10) по законам линейной алгебры
    W -= eta * (1/n * 2 * np.dot(X.T, y_pred - y))
    if i % 10 == 0:
        print(f'Iteration #{i}: W_new = {W}, MSE = {round(err,2)}')

In [ ]:
#Это вспомогательные принты ко 2ому заданию, чтобы посмотреть типы данных и размерности
print(X)
print(y_pred - y)
print(type(X))
print(type(y_pred - y))
print(np.shape(X))
print(np.shape((y_pred - y).T))

3*. Вместо того, чтобы задавать количество итераций, задайте другое условие останова алгоритма - когда веса перестают изменяться меньше определенного порога ε

In [ ]:
#Скопирую из 2го примера код сюда
n = X.shape[0]

eta = 1e-2
n_iter = 10000
min_dist = 1e-4
dist = np.inf
i = 0

W = np.array([1, 0.5])
print(f'Number of objects = {n} \
       \nLearning rate = {eta} \
       \nInitial weights = {W} \n')
#Изменю цикл на while для отсчета разности норм эпсилон и числа итерации
while dist > min_dist and i < n_iter:
    y_pred = np.dot(X, W)
    err = calc_mse(y, y_pred)
    W -= eta * (1/n * 2 * np.dot(X.T, y_pred - y))
    #Добавлю вычисление значения нового эпсилон (W++) и сравнивать разность норм
    new_W = W - eta * (1/n * 2 * np.dot(X.T, y_pred - y))
    dist = np.linalg.norm(new_W - W, ord=2)
    if i % 10 == 0:
        print(f'Iteration #{i}: W_new = {W}, MSE = {round(err,2)}')
    #И естественно нужно доавить счетчик, т.к мы ушли от цикла for
    i += 1

#В конце задания подбираю гиперпараметры которые в начале ячейки, для получения оптимального MSE